# Filter FAC, Electro jets, Density normalization, Half orbit means and pyamps

In [1]:
import pandas as pd
import numpy as np
import pickle
import My_functions_v2 as my

### Load data

In [2]:
DNS = pd.read_pickle('/home/simon/Desktop/Bachelor_project/data/pkl_files/DNS.pkl')
FAC = pd.read_pickle('/home/simon/Desktop/Bachelor_project/data/pkl_files/FAC_dual.pkl')
omni = pd.read_pickle('/home/simon/Desktop/Bachelor_project/data/pkl_files/omni.pkl')

### Filter FAC

In [3]:
FAC_filter= my.filter_FAC(FAC,dt = 10 ,Flags_B=10002,Flags_q=440404)
FAC_filter.to_pickle('/home/simon/Desktop/Bachelor_project/data/pkl_files/FAC_filter.pkl')

1542 observation out off 3802595 are removed


### Electrojets

In [4]:
Electro_jets = my.get_jets(FAC_filter) 
solar_time = DNS.Local_solar_time.reindex(Electro_jets.index,method='nearest')
Electro_jets = pd.concat([Electro_jets,solar_time], axis=1)
Electro_jets.to_pickle('/home/simon/Desktop/Bachelor_project/data/pkl_files/Electro_jets.pkl')

### Density normalization

In [5]:
normFactors = np.genfromtxt('/home/simon/Desktop/Bachelor_project/data/normalization_factors.csv', delimiter=',')

In [6]:
DNS_norm = DNS.copy()
Adjusted_density = DNS.Density*normFactors[:,0]*normFactors[:,1]
DNS_norm.loc[:,'Density'] = Adjusted_density.values
DNS_norm.to_pickle('/home/simon/Desktop/Bachelor_project/data/pkl_files/DNS_normalized.pkl')

### Half orbit means 

In [7]:
FAC_mean=my.orbit_means(FAC_filter)
FAC_power = my.orbit_means(FAC_filter,mode='power')
DNS_mean=my.orbit_means(DNS)
DNS_norm_mean = my.orbit_means(DNS_norm)

Remove incomplete orbits by considering the time 

In [8]:
N_FAC = len(FAC_mean)
N_power = len(FAC_power)
N_DNS = len(DNS_mean)
N_DNS_norm = len(DNS_norm_mean)

FAC_mean=FAC_mean.loc[abs(FAC_mean.Delta_time-FAC_mean.Delta_time.median())<360]
FAC_power=FAC_power.loc[abs(FAC_power.Delta_time-FAC_power.Delta_time.median())<360]
DNS_mean=DNS_mean.loc[abs(DNS_mean.Delta_time-DNS_mean.Delta_time.median())<360]
DNS_norm_mean=DNS_norm_mean.loc[abs(DNS_norm_mean.Delta_time-DNS_norm_mean.Delta_time.median())<360]

print("FAC: %d orbits out of %d removed" % (N_FAC-len(FAC_mean),N_FAC))
print("FAC_power: %d orbits out of %d removed" % (N_power-len(FAC_power),N_power))
print("DNS: %d orbits out of %d removed" % (N_DNS-len(DNS_mean),N_DNS))
print("DNS_norm: %d orbits out of %d removed" % (N_DNS_norm-len(DNS_norm_mean),N_DNS_norm))

FAC: 54 orbits out of 1732 removed
FAC_power: 54 orbits out of 1732 removed
DNS: 24 orbits out of 1732 removed
DNS_norm: 24 orbits out of 1732 removed


Align measument with those of DNS

In [9]:
df1 = DNS_norm_mean.Density.reindex(DNS_mean.index,method='ffill')
df1.name = 'Density_norm'

df2 = FAC_mean.FAC.reindex(DNS_mean.index,method='nearest')
df2.name = 'FAC'

df3 = FAC_power.FAC.reindex(DNS_mean.index,method='nearest')
df3.name = 'FAC_power'

Half_orbit = pd.concat([df1,df2,df3,DNS_mean.iloc[:,0:4]], axis=1)

add omni data and save 

In [10]:
omni_alligned = omni.reindex(Half_orbit.index,method='ffill')
Half_orbit = pd.concat([Half_orbit,omni_alligned], axis=1)


In [11]:
my.add_pyamps_currents(Half_orbit)

In [12]:
Half_orbit.to_pickle('/home/simon/Desktop/Bachelor_project/data/pkl_files/half_orbit_means.pkl')

In [13]:
Half_orbit.head(10)

,Density_norm,FAC,FAC_power,Density,Altitude,Orbit_nr,Hemisphere,AP_INDEX,Kp,F10_INDEX,BY_GSM,BZ_GSM,Bulk_speed,F10_Mean,PyAmps
2015-01-02 00:50:50,5.932167e-13,0.104478,0.284759,6.468488e-13,464222.287865,1.0,1.0,15.0,3.0,141.0,-1.0,-2.1,433.0,129.029218,1.769016
2015-01-02 01:37:45,7.720531e-13,0.139248,0.372618,6.987142e-13,475492.964700,1.0,-1.0,15.0,3.0,141.0,-0.6,-1.7,433.0,129.032511,2.962752
2015-01-02 02:24:40,5.577984e-13,0.113231,0.390340,6.089699e-13,464224.157119,2.0,1.0,15.0,3.0,141.0,-2.2,-2.4,434.0,129.035803,1.880936
2015-01-02 03:11:35,7.597623e-13,0.181961,0.565810,6.858343e-13,475506.057772,2.0,-1.0,4.0,1.0,141.0,-3.0,-2.3,433.0,129.039095,4.111182
2015-01-02 03:58:30,5.691719e-13,0.075313,0.165567,6.237317e-13,464035.397442,3.0,1.0,4.0,1.0,141.0,-3.0,-2.3,433.0,129.039095,1.841245
2015-01-02 04:45:25,7.214728e-13,0.084554,0.224967,6.492501e-13,475514.819681,3.0,-1.0,4.0,1.0,141.0,-3.0,-1.7,429.0,129.042387,3.673843
2015-01-02 05:32:20,5.458171e-13,0.090864,0.245008,5.960482e-13,464242.140820,4.0,1.0,4.0,1.0,141.0,-1.8,-1.1,416.0,129.045679,1.109322
2015-01-02 06:19:15,7.206711e-13,0.085828,0.200067,6.465145e-13,475520.773958,4.0,-1.0,5.0,1.3,141.0,-2.6,-1.1,416.0,129.048972,3.038630
2015-01-02 07:06:10,5.683820e-13,0.048973,0.125770,6.189799e-13,464277.002808,5.0,1.0,5.0,1.3,141.0,-5.6,-1.8,431.0,129.052264,1.965290
2015-01-02 07:53:05,7.427107e-13,0.149933,0.405583,6.669903e-13,475231.043928,5.0,-1.0,5.0,1.3,141.0,-5.6,-1.8,431.0,129.052264,4.534402
